# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from so_secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()[0]
data

{'avgTotalVolume': 70612566,
 'calculationPrice': 'close',
 'change': -2.1,
 'changePercent': -0.01425,
 'close': None,
 'closeSource': 'official',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': '15 minute delayed price',
 'highTime': 1677704400004,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 145.41,
 'iexBidSize': 248,
 'iexClose': 145.38,
 'iexCloseTime': 1677704399461,
 'iexLastUpdated': 1677704399461,
 'iexMarketPercent': 0.017101554712846165,
 'iexOpen': 146.8,
 'iexOpenTime': 1677681000097,
 'iexRealtimePrice': 145.38,
 'iexRealtimeSize': 1,
 'iexVolume': 948777,
 'lastTradeTime': 1677704399998,
 'latestPrice': 145.31,
 'latestSource': 'Close',
 'latestTime': 'March 1, 2023',
 'latestUpdate': 1677704400806,
 'latestVolume': None,
 'low': None,
 'lowSource': '15 mi

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
data['ytdChange']

0.11662066480950349

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
requests.get(f'https://api.iex.cloud/v1/data/CORE/QUOTE,STATS/AAPL?token=pk_a75112fb0e604c06bf85e484fc778432').json()

[{'avgTotalVolume': 70612566,
  'calculationPrice': 'close',
  'change': -2.1,
  'changePercent': -0.01425,
  'close': None,
  'closeSource': 'official',
  'closeTime': None,
  'companyName': 'Apple Inc',
  'currency': 'USD',
  'delayedPrice': None,
  'delayedPriceTime': None,
  'extendedChange': None,
  'extendedChangePercent': None,
  'extendedPrice': None,
  'extendedPriceTime': None,
  'high': None,
  'highSource': '15 minute delayed price',
  'highTime': 1677704400004,
  'iexAskPrice': 0,
  'iexAskSize': 0,
  'iexBidPrice': 145.41,
  'iexBidSize': 248,
  'iexClose': 145.38,
  'iexCloseTime': 1677704399461,
  'iexLastUpdated': 1677704399461,
  'iexMarketPercent': 0.017101554712846165,
  'iexOpen': 146.8,
  'iexOpenTime': 1677681000097,
  'iexRealtimePrice': 145.38,
  'iexRealtimeSize': 1,
  'iexVolume': 948777,
  'lastTradeTime': 1677704399998,
  'latestPrice': 145.31,
  'latestSource': 'Close',
  'latestTime': 'March 1, 2023',
  'latestUpdate': 1677704400806,
  'latestVolume': Non

In [8]:
data

{'avgTotalVolume': 70612566,
 'calculationPrice': 'close',
 'change': -2.1,
 'changePercent': -0.01425,
 'close': None,
 'closeSource': 'official',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': '15 minute delayed price',
 'highTime': 1677704400004,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 145.41,
 'iexBidSize': 248,
 'iexClose': 145.38,
 'iexCloseTime': 1677704399461,
 'iexLastUpdated': 1677704399461,
 'iexMarketPercent': 0.017101554712846165,
 'iexOpen': 146.8,
 'iexOpenTime': 1677681000097,
 'iexRealtimePrice': 145.38,
 'iexRealtimeSize': 1,
 'iexVolume': 948777,
 'lastTradeTime': 1677704399998,
 'latestPrice': 145.31,
 'latestSource': 'Close',
 'latestTime': 'March 1, 2023',
 'latestUpdate': 1677704400806,
 'latestVolume': None,
 'low': None,
 'lowSource': '15 mi

In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote,stats/{symbol_string}?token=pk_a75112fb0e604c06bf85e484fc778432'
    data = requests.get(batch_api_call_url).json()
    for i in range(0, len(data)//2):
        stock_quote = data[i]
        stock_stats = data[i + (len(data)//2)]
        my_series = pd.Series(
        [
            stock_quote['symbol'],
            stock_quote['latestPrice'],
            stock_stats['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns
        )
        final_dataframe = pd.concat([final_dataframe, my_series.to_frame().T], ignore_index=True)

In [10]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,137.51,0.062806,N/A
1,AAL,16.03,-0.10882,N/A
2,AAP,138.82,-0.300243,N/A
3,AAPL,145.31,-0.103518,N/A
4,ABBV,155.27,0.06007,N/A
...,...,...,...,...
95,CINF,119.92,-0.003719,N/A
96,CL,72.55,-0.040645,N/A
97,CLX,154.69,0.078194,N/A
98,CMA,70.68,-0.236387,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [11]:
final_dataframe.sort_values("One-Year Price Return", ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,77,CAH,75.98,0.477079,N/A
1,54,AZO,2456.37,0.362501,N/A
2,23,ALB,252.64,0.30841,N/A
3,62,BIIB,272.21,0.302054,N/A
4,79,CAT,248.67,0.292107,N/A
5,39,ANTM,482.58,0.281524,N/A
6,94,CI,288.75,0.27737,N/A
7,85,CDNS,191.43,0.266854,N/A
8,16,AES,24.63,0.232153,N/A
9,43,APD,288.28,0.198892,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [12]:
def portfolio_input():
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        return float(portfolio_size)
    except ValueError:
        print("That's not a number! \nPlease try again:")
        return portfolio_input()
portfolio_size = portfolio_input()

Enter the size of your portfolio:1000000


In [13]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,77,CAH,75.98,0.477079,263
1,54,AZO,2456.37,0.362501,8
2,23,ALB,252.64,0.30841,79
3,62,BIIB,272.21,0.302054,73
4,79,CAT,248.67,0.292107,80
5,39,ANTM,482.58,0.281524,41
6,94,CI,288.75,0.27737,69
7,85,CDNS,191.43,0.266854,104
8,16,AES,24.63,0.232153,812
9,43,APD,288.28,0.198892,69


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [79]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote,stats/{symbol_string}?token=pk_a75112fb0e604c06bf85e484fc778432'
    data = requests.get(batch_api_call_url).json()
    for i in range(0, len(data)//2):
        stock_quote = data[i]
        stock_stats = data[i + (len(data)//2)]
        my_series = pd.Series(
        [
            stock_quote['symbol'],
            stock_quote['latestPrice'],
            'N/A',
            stock_stats['year1ChangePercent'],
            'N/A',
            stock_stats['month6ChangePercent'],
            'N/A',
            stock_stats['month3ChangePercent'],
            'N/A',
            stock_stats['month1ChangePercent'],
            'N/A',
            'N/A'
        ],
        index = hqm_columns
        )
        hqm_dataframe = pd.concat([hqm_dataframe, my_series.to_frame().T], ignore_index=True)

In [80]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.51,N/A,0.062806,N/A,0.041507,N/A,-0.099066,N/A,-0.093477,N/A,N/A
1,AAL,16.03,N/A,-0.10882,N/A,0.079057,N/A,0.073103,N/A,-0.04305,N/A,N/A
2,AAP,138.82,N/A,-0.300243,N/A,-0.217824,N/A,-0.064552,N/A,-0.068808,N/A,N/A
3,AAPL,145.31,N/A,-0.103518,N/A,-0.133309,N/A,-0.006688,N/A,0.035768,N/A,N/A
4,ABBV,155.27,N/A,0.06007,N/A,0.117708,N/A,-0.034196,N/A,0.03329,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.2,N/A,0.040014,N/A,0.096011,N/A,0.002371,N/A,-0.018344,N/A,N/A
501,ZBH,122.08,N/A,-0.023494,N/A,0.100659,N/A,0.031228,N/A,-0.017614,N/A,N/A
502,ZBRA,302.34,N/A,-0.301744,N/A,-0.115769,N/A,0.07363,N/A,-0.060051,N/A,N/A
503,ZION,50.44,N/A,-0.27042,N/A,-0.108761,N/A,-0.020986,N/A,-0.026068,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [81]:
mask = hqm_dataframe['One-Year Price Return'].isnull() \
            | hqm_dataframe['Six-Month Price Return'].isnull() \
            | hqm_dataframe['Three-Month Price Return'].isnull() \
            | hqm_dataframe['One-Month Price Return'].isnull()

hqm_dataframe = hqm_dataframe.drop(hqm_dataframe[mask].index)

In [82]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.51,N/A,0.062806,69.660679,0.041507,62.874251,-0.099066,13.972056,-0.093477,9.580838,N/A
1,AAL,16.03,N/A,-0.10882,33.133733,0.079057,72.45509,0.073103,79.840319,-0.04305,31.137725,N/A
2,AAP,138.82,N/A,-0.300243,9.780439,-0.217824,7.185629,-0.064552,23.752495,-0.068808,19.760479,N/A
3,AAPL,145.31,N/A,-0.103518,34.530938,-0.133309,18.163673,-0.006688,50.499002,0.035768,82.43513,N/A
4,ABBV,155.27,N/A,0.06007,68.862275,0.117708,81.437126,-0.034196,37.52495,0.03329,81.437126,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.2,N/A,0.040014,65.06986,0.096011,75.848303,0.002371,55.688623,-0.018344,50.299401,N/A
501,ZBH,122.08,N/A,-0.023494,50.0998,0.100659,77.644711,0.031228,68.463074,-0.017614,51.497006,N/A
502,ZBRA,302.34,N/A,-0.301744,9.580838,-0.115769,21.956088,0.07363,80.03992,-0.060051,24.550898,N/A
503,ZION,50.44,N/A,-0.27042,11.776447,-0.108761,24.750499,-0.020986,42.315369,-0.026068,42.914172,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [83]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.51,N/A,0.062806,69.660679,0.041507,62.874251,-0.099066,13.972056,-0.093477,9.580838,39.021956
1,AAL,16.03,N/A,-0.10882,33.133733,0.079057,72.45509,0.073103,79.840319,-0.04305,31.137725,54.141717
2,AAP,138.82,N/A,-0.300243,9.780439,-0.217824,7.185629,-0.064552,23.752495,-0.068808,19.760479,15.11976
3,AAPL,145.31,N/A,-0.103518,34.530938,-0.133309,18.163673,-0.006688,50.499002,0.035768,82.43513,46.407186
4,ABBV,155.27,N/A,0.06007,68.862275,0.117708,81.437126,-0.034196,37.52495,0.03329,81.437126,67.315369
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.2,N/A,0.040014,65.06986,0.096011,75.848303,0.002371,55.688623,-0.018344,50.299401,61.726547
501,ZBH,122.08,N/A,-0.023494,50.0998,0.100659,77.644711,0.031228,68.463074,-0.017614,51.497006,61.926148
502,ZBRA,302.34,N/A,-0.301744,9.580838,-0.115769,21.956088,0.07363,80.03992,-0.060051,24.550898,34.031936
503,ZION,50.44,N/A,-0.27042,11.776447,-0.108761,24.750499,-0.020986,42.315369,-0.026068,42.914172,30.439122


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [84]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True, drop = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FTI,15.42,N/A,1.238235,99.800399,0.745413,99.800399,0.257851,97.60479,0.118295,97.40519,98.652695
1,URI,467.8,N/A,0.428672,96.606786,0.414098,99.001996,0.258678,97.804391,0.151405,98.602794,98.003992
2,LB,79.92,N/A,2.281287,100.0,0.815831,100.0,0.215421,96.606786,0.079271,93.812375,97.60479
3,COTY,11.31,N/A,0.221249,90.818363,0.364749,98.403194,0.470976,99.800399,0.13198,98.203593,96.806387
4,RE,390.84,N/A,0.306912,94.610778,0.402111,98.802395,0.154746,92.814371,0.100701,96.407186,95.658683
5,COG,22.25,N/A,0.340483,95.409182,0.180873,89.820359,0.255629,97.205589,0.249298,99.800399,95.558882
6,WYNN,112.22,N/A,0.202347,89.021956,0.624788,99.600798,0.408381,99.401198,0.070393,92.015968,95.00998
7,BWA,50.65,N/A,0.195945,88.023952,0.2913,96.007984,0.183303,95.00998,0.109922,96.806387,93.962076
8,PH,353.35,N/A,0.19436,87.42515,0.211165,92.814371,0.141555,92.41517,0.126209,98.003992,92.664671
9,GWW,671.01,N/A,0.414268,96.407186,0.148109,86.027944,0.101915,85.628743,0.189305,99.001996,91.766467


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [85]:
portfolio_input()

Enter the size of your portfolio:1000000


1000000.0

In [86]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe)):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

hqm_dataframe

C:\Users\freel\AppData\Local\Temp\ipykernel_11588\2596024171.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FTI,15.42,1297,1.238235,99.800399,0.745413,99.800399,0.257851,97.60479,0.118295,97.40519,98.652695
1,URI,467.8,42,0.428672,96.606786,0.414098,99.001996,0.258678,97.804391,0.151405,98.602794,98.003992
2,LB,79.92,250,2.281287,100.0,0.815831,100.0,0.215421,96.606786,0.079271,93.812375,97.60479
3,COTY,11.31,1768,0.221249,90.818363,0.364749,98.403194,0.470976,99.800399,0.13198,98.203593,96.806387
4,RE,390.84,51,0.306912,94.610778,0.402111,98.802395,0.154746,92.814371,0.100701,96.407186,95.658683
5,COG,22.25,898,0.340483,95.409182,0.180873,89.820359,0.255629,97.205589,0.249298,99.800399,95.558882
6,WYNN,112.22,178,0.202347,89.021956,0.624788,99.600798,0.408381,99.401198,0.070393,92.015968,95.00998
7,BWA,50.65,394,0.195945,88.023952,0.2913,96.007984,0.183303,95.00998,0.109922,96.806387,93.962076
8,PH,353.35,56,0.19436,87.42515,0.211165,92.814371,0.141555,92.41517,0.126209,98.003992,92.664671
9,GWW,671.01,29,0.414268,96.407186,0.148109,86.027944,0.101915,85.628743,0.189305,99.001996,91.766467


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [88]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [89]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [90]:
column_formats ={
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', integer_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', integer_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', integer_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', integer_template],
    'L': ['HQM Score', integer_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [91]:
writer.close()